In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/CS535/HW2/dataset.csv')

In [ ]:
data.head()

,file_name_list,speakers,visual_features,acoustic_features,lexical_features,emotion_labels
0,Ses01F_impro01_F001,F01,/features/visual_features/Session1/Ses01F_impr...,/features/acoustic_features/Session1/Ses01F_im...,/features/lexical_features/Session1/Ses01F_imp...,3
1,Ses01F_impro01_M011,M01,/features/visual_features/Session1/Ses01F_impr...,/features/acoustic_features/Session1/Ses01F_im...,/features/lexical_features/Session1/Ses01F_imp...,0
2,Ses01F_impro02_F002,F01,/features/visual_features/Session1/Ses01F_impr...,/features/acoustic_features/Session1/Ses01F_im...,/features/lexical_features/Session1/Ses01F_imp...,1
3,Ses01F_impro02_F003,F01,/features/visual_features/Session1/Ses01F_impr...,/features/acoustic_features/Session1/Ses01F_im...,/features/lexical_features/Session1/Ses01F_imp...,3
4,Ses01F_impro02_F004,F01,/features/visual_features/Session1/Ses01F_impr...,/features/acoustic_features/Session1/Ses01F_im...,/features/lexical_features/Session1/Ses01F_imp...,1


In [ ]:
len(data)

1336

In [ ]:
from collections import Counter
Counter(data['emotion_labels'])

Counter({3: 520, 0: 328, 1: 308, 2: 180})

In [ ]:
def build_paths_to_file(df_with_paths, specific_feature):
  features = df_with_paths.loc[0:, ['file_name_list', 'speakers', specific_feature, 'emotion_labels']]
  features_path = features.loc[0:, specific_feature]
  features["file_with_path"] = features_path
  list_of_features = list(features["file_with_path"])

  return list_of_features, features

In [ ]:
def reduce_temporal_dimension(features_paths):

    reduced_features = []

    for row in range(len(features_paths)):
        load_features_file = np.load('/content/drive/MyDrive/CS535/HW2' + features_paths[row])
        resampled = np.mean(load_features_file, axis=0)
        reduced_features.append(resampled)

    return reduced_features

In [ ]:
def reduced_dataset_prep(specific_feature):
  features_paths, features_with_y = build_paths_to_file(data, specific_feature)
  x = reduce_temporal_dimension(features_paths)
  x = pd.DataFrame(x)
  y = features_with_y['emotion_labels']
  y = pd.DataFrame(y)
  return x, y

In [ ]:
visual_x, visual_y = reduced_dataset_prep('visual_features')
visual_x.to_csv('/content/drive/MyDrive/CS535/HW2/visual_x.csv', index=False)
visual_y.to_csv('/content/drive/MyDrive/CS535/HW2/visual_y.csv', index=False)

In [ ]:
audio_x, audio_y = reduced_dataset_prep('acoustic_features')
audio_x.to_csv('/content/drive/MyDrive/CS535/HW2/audio_x.csv', index=False)
audio_y.to_csv('/content/drive/MyDrive/CS535/HW2/audio_y.csv', index=False)

In [ ]:
import os
def load_text_features(text_features_paths, ys):

    loaded_text_features = []
    true_labels = []

    for row in range(len(text_features_paths)):
      load_text_features_file = np.load('/content/drive/MyDrive/CS535/HW2' + text_features_paths[row])
      loaded_text_features.append(load_text_features_file)
      true_labels.append(ys[row])
    return loaded_text_features, true_labels

In [ ]:
text_features_paths, text_features_with_y = build_paths_to_file(data, 'lexical_features')
text_ys_series = text_features_with_y.loc[0:, 'emotion_labels']
text_x, text_y = load_text_features(text_features_paths, text_ys_series)
text_x = pd.DataFrame(text_x)
text_y = pd.DataFrame(text_y)
text_x.to_csv('/content/drive/MyDrive/CS535/HW2/text_x.csv', index=False)
text_y.to_csv('/content/drive/MyDrive/CS535/HW2/text_y.csv', index=False)